# Association rules - framework


[1] Agrawal, R.; Imieliński, T.; Swami, A. (1993). "Mining association rules between sets of items in large databases". Proceedings of the 1993 ACM SIGMOD international conference on Management of data - SIGMOD '93. p. 207.

The dataset in the association rules framework is defined as a set of `transactions` $t$. The entire dataset is referred to as `database` $D = \{t_1, t_2, ..., t_n\}$ and the attributes are called `items` $I = \{i_1,i_2, ..., i_m\}$. This terminology stems from the analysis of supermaket transactions, which was introduced in the original paper, which discusses the problem of association rule mining[1]. For now we will use this limitted interpretation, and then we will generalize it. Transactions are independent of each other and are comprised of sets of purchased items. Take a look at the example:

   | Transaction | Milk | Cookies | Bread |
   | --- | --- | --- | --- | 
   | #1 | 1 | 1 | 1 |
   | #2 | 1 | 1 | 0 |
   | #3 | 0 | 1 | 0 |
   | #4 | 0 | 1 | 1 |

Note that the values of items are binary. Either milk was bought, or not.


In this framework a rule is defined as 

$$ X \rightarrow Y : X,Y \subseteq I $$

 Basically a rule that states: "if a user bought X (where X is a subset of items), then he also bought Y (Y is also a subset of items)"(In the original work, $|Y|=1$). In this framework, $X$ is called antecedent and $Y$ is called consequent. Let's take a look at the example again. In this case the following rule is always true:


$$ \{Milk\} \rightarrow \{Cookies\}$$

It doesn't mean that 

$$ \{Cookies\} \rightarrow \{Milk\}$$

is always true.

Let's load a sample database.


In [16]:
import pandas as pd
marketbasket = pd.read_csv('marketbasket.csv')
marketbasket = marketbasket.rename(columns = lambda x : x.strip())
print(marketbasket.columns)
marketbasket

Index(['Hair Conditioner', 'Lemons', 'Standard coffee', 'Frozen Chicken Wings',
       '98pct. Fat Free Hamburger', 'Sugar Cookies', 'Onions', 'Deli Ham',
       'Dishwasher Detergent', 'Beets',
       ...
       'Lollipops', 'Plain White Bread', 'Blueberry Yogurt',
       'Frozen Chicken Thighs', 'Mixed Vegetables', 'Souring Pads',
       'Tuna Spread', 'Toilet Paper', 'White Wine', 'Columbian Coffee'],
      dtype='object', length=255)


,Hair Conditioner,Lemons,Standard coffee,Frozen Chicken Wings,98pct. Fat Free Hamburger,Sugar Cookies,Onions,Deli Ham,Dishwasher Detergent,Beets,...,Lollipops,Plain White Bread,Blueberry Yogurt,Frozen Chicken Thighs,Mixed Vegetables,Souring Pads,Tuna Spread,Toilet Paper,White Wine,Columbian Coffee
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1358,0,0,0,1,1,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1359,1,0,0,0,1,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0


In this database we have a total of 1361 transactions and 255 items. We are also going to create a very small toy dataset to illustrate the ideas (the same dataset is used in the [Wikipedia article](https://en.wikipedia.org/wiki/Association_rule_learning) on the Association Rules).



In [2]:
import pandas as pd

data = {
    'Milk' : [1,0,0,1,0],
    'Bread' : [1,0,0,1,1],
    'Butter' : [0,1,0,1,0],
    'Beer' : [0,0,1,0,0],
    'Diapers' : [0,0,1,0,0],
    'Egg' : [0,1,0,1,0],
    'Fruit' : [1,1,0,1,0]
}

df = pd.DataFrame(data)
df

,Milk,Bread,Butter,Beer,Diapers,Egg,Fruit
0,1,1,0,0,0,0,1
1,0,0,1,0,0,1,1
2,0,0,0,1,1,0,0
3,1,1,1,0,0,1,1
4,0,1,0,0,0,0,0


A rule becomes an association rule when it surpasses a given threshold in key metrics. By far the most common combination of the metrics, which determine thresholds for an association rules are support and confidence.


## Support

Support is given as a frequency of an itemset in the entire database. For example, in this database, the itemset $\{fruit\}$ appeared 3 times out of a total of 5 transactions, which means that support for this itemset is equal to $supp(\{fruit\}) = \frac{3}{5}=0.6$. Fruit in combination with bread appeared 2 times in the entire dataset, so $supp(\{fruit, bread\}) = \frac{2}{5} = 0.4$. We can formalize the support as 

$$supp(X) = P(\bigcap_{A_i \in X}A_i) = \frac{\textrm{number of transactions, with an item for all categories in the subset X}}{\textrm{total number of transactions}}$$

Let's calculate support for each one-item subset in the toy dataset.


In [3]:
import numpy as np
subsets = df.columns.values.reshape(7,1)

support = pd.DataFrame(subsets, columns = ['item'])
support['support'] = support.apply(lambda x : df[x['item']].sum()/df.shape[0], axis=1)
support

,item,support
0,Milk,0.4
1,Bread,0.6
2,Butter,0.4
3,Beer,0.2
4,Diapers,0.2
5,Egg,0.4
6,Fruit,0.6


Let's now calculate the support for itemsets, which consist of two items.

In [4]:
import numpy as np
subsets = df.columns.values.reshape(-1,1)
#take all columns from the dataset
items = pd.DataFrame(subsets, columns = ['item'])


 | item |
 | --- |
 | bread |
 | butter |
 | diapers |
 | egg |
 | fruit |
 | milk |

In [5]:
#create a cross product for all column names
sc = items.merge(items, how='cross')


 | item_x\item_y | beer | bread | butter | diapers | egg | fruit | milk |
 | --- |  --- |  --- |  --- |  --- |  --- |  --- | --- |
  | beer | (beer, beer) | (beer,bread) |  (beer, butter) | (beer,diapers) | (beer, egg) |  (beer, fruit) | (beer,milk) |
 | bread | (bread, beer) | (bread,bread) |  (bread, butter) | (bread,diapers) | (bread, egg) |  (bread, fruit) | (bread,milk) |
 | butter | (butter, beer) | (butter,bread) |  (butter, butter) | (butter, diapers) | (butter,egg) |  (butter, fruit) |  (butter, milk) |
 | diapers | (diapers, beer) | (diapers, bread) | (diapers, butter) | (diapers, diapers) |  (diapers,egg) | (diapers, fruit) | (diapers, milk) |
 | egg | (egg, beer) |(egg, bread) | (egg, butter) | (egg, diapers) |  (egg,egg) | (egg, fruit) | (egg, milk) |
 | fruit | (fruit, beer) |(fruit, bread) | (fruit, butter) | (fruit, diapers) |  (fruit,egg) | (fruit, fruit) | (fruit, milk) |
 | milk | (milk, beer) |(milk, bread) | (milk, butter) | (milk, diapers) |  (milk,egg) | (milk, fruit) | (milk, milk) |

In [6]:
#take only the lower triangle of the cross product
support_two_items = sc[(sc['item_x'] > sc['item_y'])][['item_x','item_y']].reset_index()

 | item_x\item_y | beer | bread | butter | diapers | egg | fruit | milk |
 | --- |  --- |  --- |  --- |  --- |  --- |  --- | --- |
  | beer | 
 | bread | (bread, beer) 
 | butter | (butter, beer) | (butter,bread) 
 | diapers | (diapers, beer) | (diapers, bread) | (diapers, butter) 
 | egg | (egg, beer) |(egg, bread) | (egg, butter) | (egg, diapers) 
 | fruit | (fruit, beer) |(fruit, bread) | (fruit, butter) | (fruit, diapers) 
 | milk | (milk, beer) |(milk, bread) | (milk, butter) | (milk, diapers) |  (milk,egg) | (milk, fruit) 

In [7]:
#calculate the support for each pair
support_two_items['support'] = support_two_items.apply(lambda x : df[(df[x['item_x']] == 1) & (df[x['item_y']] == 1)].shape[0]/df.shape[0], axis=1)
support_two_items

,index,item_x,item_y,support
0,1,Milk,Bread,0.4
1,2,Milk,Butter,0.2
2,3,Milk,Beer,0.0
3,4,Milk,Diapers,0.0
4,5,Milk,Egg,0.2
5,6,Milk,Fruit,0.4
6,10,Bread,Beer,0.0
7,15,Butter,Bread,0.2
8,17,Butter,Beer,0.0
9,29,Diapers,Bread,0.0



Before we go any further let's discuss one more thing. The definition of support for a rule $X \rightarrow Y$ is given by $supp(X\cup Y)$. In terms of support the antecedent and consequent itemsets are treated equally, as $supp(X\cup Y) = supp(Y \cup X)$. 

## Confidence

While support tells us, how frequently the rule appears in the data, confidence tells us how precise is the rule. Confidence is given by 

$$conf(X \rightarrow Y) = \frac{supp(X \cup Y)}{supp(X)}$$

Notice the similarity between the definition of confidence and Bayes theorem. It's not accidental, technically confidence is implemented as a conditional probability. We can use the previously calculated values of support to calculate the confidence for itemsets, which consist of one item.


In [8]:
confidence = support_two_items.apply(lambda x: 
                                        {'antecedent' : x['item_x'], 
                                         'consequent' : x['item_y'], 
                                         'support' : x['support'],
                                         'confidence': x['support'] / support[(support['item'] == x['item_x'])]['support'].values[0]}, axis=1)

pd.DataFrame(confidence.tolist())

,antecedent,consequent,support,confidence
0,Milk,Bread,0.4,1.000000
1,Milk,Butter,0.2,0.500000
2,Milk,Beer,0.0,0.000000
3,Milk,Diapers,0.0,0.000000
4,Milk,Egg,0.2,0.500000
5,Milk,Fruit,0.4,1.000000
6,Bread,Beer,0.0,0.000000
7,Butter,Bread,0.2,0.500000
8,Butter,Beer,0.0,0.000000
9,Diapers,Bread,0.0,0.000000



# Frequent Itemsets

We have evaluated the values of support and confidence for itemsets of a size one. Now we need to expand our analysis to all possible subsets of items. I hope that you can clearly see the problem. If our dataset contains a lot of items, the number of possible combinations gets really large, really quickly (as quickly as $2^n$ for $n$ items). This problem is solved by algorithms of discovering frequent datasets.

Before we solve the problem using algorithms of discovering frequent datasets let's implement the brute force algoritm. We are going to evaluate its performance in the context of frequently used, common algorithms for discovering frequent datasets.



In [9]:
def calculate_c_level_permutations(previous_permutations, columns, level):
  columns = pd.DataFrame(columns.reshape(-1,1))
  previous_permutations = pd.DataFrame(previous_permutations.reshape(-1,level))
  cartesian = previous_permutations.merge(columns, how='cross')
  filtered_cartesian = cartesian[(cartesian.iloc[:,0:-1].min(axis=1) > cartesian.iloc[:,-1])]
  return filtered_cartesian.to_numpy()

def get_all_permutations(columns):
  level = 1
  c = calculate_c_level_permutations(columns,columns,1)
  all = []
  for item in columns:
    all.append(np.array([item], dtype=object))
  for item in c:
    all.append(item)

  while(len(c) > 0):
    level += 1
    c = calculate_c_level_permutations(c,columns,level)
    for item in c:
      all.append(item)
  return all
def calculate_support(permutation, df):
  permutation = [p for p in permutation.values if p is not None]
  return df[permutation].prod(axis=1).sum()/df.shape[0]
columns = df.columns.values
permutations = get_all_permutations(columns)
support_all = pd.DataFrame(permutations)
support_all['support'] = support_all.apply(lambda x: calculate_support(x,df),axis=1)
support_all

,0,1,2,3,4,5,6,support
0,Milk,None,None,None,None,None,None,0.4
1,Bread,None,None,None,None,None,None,0.6
2,Butter,None,None,None,None,None,None,0.4
3,Beer,None,None,None,None,None,None,0.2
4,Diapers,None,None,None,None,None,None,0.2
...,...,...,...,...,...,...,...,...
122,Milk,Fruit,Egg,Diapers,Bread,Beer,None,0.0
123,Milk,Fruit,Egg,Diapers,Butter,Bread,None,0.0
124,Milk,Fruit,Egg,Diapers,Butter,Beer,None,0.0
125,Fruit,Egg,Diapers,Butter,Bread,Beer,None,0.0


We are not even going to test the brute force algorithm on the marketbasket dataset. It would evaluate $2^255$ permutations, which is waaay too much. We are just going to test the implementation on the first 12 columns.

In [17]:
%%time

columns = marketbasket.columns.values[:12]
permutations = get_all_permutations(columns)
support_all = pd.DataFrame(permutations)
support_all['support'] = support_all.apply(lambda x: calculate_support(x,marketbasket),axis=1)


CPU times: user 980 ms, sys: 7.34 ms, total: 987 ms
Wall time: 987 ms



## Apriori

The most common algorithm of discovering frequent datasets is called Apriori. Note that the algorithm is indifferent to used metrics, or a set of metrics used in the algorithm (in this case, we are using support). We could use this algorithm with any set of measures, as long as for every used metrics

$$B \subseteq A ⇒ m(A) \ge m(B), $$

If $B$ is a subset of $A$, than a value of metric $m$ for a set $A$ is greater or equal than value of this metric for a set $B$. Note that this is true for support, and it's not true for confidence. Hence, the algorithm is used for the support metric, and once frequent itemsets are obtained, the confidence is calculated for each of them. If a rule doesn't fulfill the threshold for confidence it's removed after discovering frequent itemsets.

In fact the Apriori algorithm has complexity of $\mathcal{O}(2^n)$ (both time and space complexity). At first glance it wouldn't perform much better than the brute force, however the average computation time for this algorithm is much lower than the exhaustive algorithm. Especially for sparse datasets.

The implementation of Apriori algorithm is very similar to our current brute-force implementation. The algorithm goes like this:

Input:
   - threshold t
   - metric m

Output:
   - Frequent sets

Algorithm:

  1. Frequent sets = [ ]
  2. The initial set of candidate itemsets consists of subsets of a size 1.
  3. Prune the set of candidates. Remove the itemsets with m(itemset) < t.
  4. Append pruned set to Frequent sets
  5. While the size of pruned set is greater than 0.


      a. Create candidate itemset, by appending each item to every entry in the pruned set

      a1. Remove the entries with repeated items from the candidate itemset (e.g. ['Milk','Bread','Milk'] should be removed)

      a2. Remove duplicates from the candidate itemset (e.g. one of ['Milk','Bread'] or ['Bread','Milk'] should be removed)

      b. Clear the pruned set.

      c. Prune the set of candidates. Remove the itemsets with m(itemset) < t.

      d. Append pruned set to Frequent sets.
  6. Return Frequent sets

## Exercise/homework:

Implement your version the apriori algorithm, it doesn't have to be very efficient. You can use the 'support' metric to prune the candidate itemsets. Threshold should be a parameter of the implemented function.

Compare the performance of two implementations (brute-force and apriori) using the large dataset imported at the beginning of this notebook.
You might notice that the method of computing support and the list of candidates in the brute force example is inefficient. Feel free to work on the efficiency of those methods.


In [75]:
def apriori(df, threshold):
    frequent_sets = []
    subsets = df.columns.values
    support = pd.DataFrame(subsets, columns = ['item'])
    support['support'] = support.apply(lambda x : df[x['item']].sum()/df.shape[0], axis=1)
    pruned = support[support['support'] >= threshold]
    pruned = pd.DataFrame([{'item' : [item], 'support' : support} for item, support in zip(pruned['item'], pruned['support'])])
    frequent_sets.append(pruned)
    while pruned.shape[0] > 0:
        new_pruned = []
        for index, row in pruned.iterrows():
            for item in df.columns.values:
                if item in row['item']:
                    continue
                candidate = [*row['item'], item]
                if len(candidate) != len(set(candidate)):
                    continue
                candidate = sorted(candidate)
                support = df[candidate].prod(axis=1).sum()/df.shape[0]
                if support >= threshold:
                    new_pruned.append({'item' : candidate, 'support' : support})
        pruned = pd.DataFrame(new_pruned)
        frequent_sets.append(pruned)
        
    return frequent_sets

    

In [81]:
%%time

columns = marketbasket.columns.values[:15]
apriori(marketbasket[columns], 0.03)

CPU times: user 41.8 ms, sys: 1.91 ms, total: 43.7 ms
Wall time: 42.2 ms


[                          item   support
 0           [Hair Conditioner]  0.058780
 1                     [Lemons]  0.030125
 2  [98pct. Fat Free Hamburger]  0.093314
 3              [Sugar Cookies]  0.055107
 4                     [Onions]  0.080088
 5             [Cottage Cheese]  0.047759,
 Empty DataFrame
 Columns: []
 Index: []]

In [82]:
%%time

columns = marketbasket.columns.values
apriori(marketbasket[columns], 0.03)

CPU times: user 25.5 s, sys: 83.1 ms, total: 25.5 s
Wall time: 25.6 s


[                            item   support
 0             [Hair Conditioner]  0.058780
 1                       [Lemons]  0.030125
 2    [98pct. Fat Free Hamburger]  0.093314
 3                [Sugar Cookies]  0.055107
 4                       [Onions]  0.080088
 ..                           ...       ...
 125                  [Lollipops]  0.043350
 126      [Frozen Chicken Thighs]  0.030125
 127           [Mixed Vegetables]  0.035268
 128               [Toilet Paper]  0.074210
 129           [Columbian Coffee]  0.030125
 
 [130 rows x 2 columns],
                                           item   support
 0                     [Eggs, Hair Conditioner]  0.030860
 1              [Hair Conditioner, White Bread]  0.030860
 2    [98pct. Fat Free Hamburger, Potato Chips]  0.043350
 3        [98pct. Fat Free Hamburger, Hot Dogs]  0.032329
 4      [2pct. Milk, 98pct. Fat Free Hamburger]  0.038207
 ..                                         ...       ...
 223                 [2pct. Milk, Toile

 ## A ready, efficient implementation of Apriori.

To complete the exercise we will need the `mlextend` library since `scikit-learn` does not provide any tools for frequent itemset or association rule discovery.

In [13]:

from mlxtend.frequent_patterns import apriori



In [14]:
%time


frequent_itemsets = apriori(marketbasket, min_support=0.03, use_colnames=True)
print(frequent_itemsets)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 8.11 µs
      support                                itemsets
0    0.058780                      (Hair Conditioner)
1    0.030125                                (Lemons)
2    0.093314             (98pct. Fat Free Hamburger)
3    0.055107                         (Sugar Cookies)
4    0.080088                                (Onions)
..        ...                                     ...
255  0.033064   (Toothpaste, White Bread, 2pct. Milk)
256  0.031594  (White Bread, 2pct. Milk, Wheat Bread)
257  0.030125               (White Bread, Cola, Eggs)
258  0.032329         (Toothpaste, White Bread, Eggs)
259  0.030125        (White Bread, Eggs, Wheat Bread)

[260 rows x 2 columns]


/Users/luncenok/Studia/sem4/DM/.venv/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Now that we have the frequent datasets discovery implemented, we can apply the confidence threshold to the discovered frequent sets.

In [15]:

from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, 
                  metric='confidence', 
                  min_threshold=0.8)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(Toothpaste, Potato Chips)",(White Bread),0.037472,0.119030,0.030125,0.803922,6.753934,0.025665,4.492946,0.885105
1,"(Sweet Relish, Hot Dog Buns)",(Hot Dogs),0.036003,0.092579,0.030125,0.836735,9.038063,0.026792,5.557954,0.922572


This is the first part of the Association rules/Frequent Datasets notebook. Next week we're going to get familiar with the ECLAT and FP-growth algorithms as well as a ready implementation of all three algorithms in Python in the mlxtend package.